## Assigning circles to weather stations
### Purpose
Using a custom table created from uploading the CSV to Big Query (this table is called `cleaned_bird_counts_gstorage`) a join is done with the view that contains the flatten data.

### Author: 
Francisco Vannini
### Date: 
2020-04-02
### Update Date: 
2020-04-02

### Inputs
<ol>
<li> Google credential auth JSON </li>
<li> noaa_from_1900_to_present view in BQ</li>
<li> flatten_noaa_from_1900_to_present in BQ</li>
<li> cleaned_bird_count data</li>
</ol>

### Output Files
This notebook produces <strong>1.1-circles-to-many-noaa-stations-usa-weather-data-[data_this_process_was_run].csv.gzip</strong>. This data contains non-empty weather measurements for the NOAA stations that are in close proximity (using geohashes) of our CDC bird count. 

## Steps or Proceedures in the notebook
This notebook creates a query that interlaces the CDC bird count data, matches it with NOAA stations in close proximity with this station and then extracts the NOAA station weather measurements pertinenet to the dates. After the data is extracted the rows that have a NULL value of "temp_min" are pruned AND only USA weather measurements included.

## Where the Data will Be Saved 
This script produces data at the level where this notebook is located.

In [14]:
!pip install google-cloud-bigquery==0.24.0
#!pip install pandas

In [15]:
# Imports
import os
from datetime import datetime
from google.cloud import bigquery
import pandas as pd
import pandas

In [10]:
# Set Up the Enviroment 

# The path to your json credentials file. Replace with your corresponding file.
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "your_path_to_google_auth_keys.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../apicred/BirdProject-2020-d9b935674747.json"

# Used to classify the name 
time_now = datetime.today().strftime('%Y%m%d%H%M%S')

client = bigquery.Client()
project = 'birdproject-2020'
source_dataset_id = 'audubon_cdc'
# source_table_id = 'us_states'
shared_dataset_ref = client.dataset(source_dataset_id)

## Load in the Most Recent Data File 
THIS IS NOT REQUIRED -- But It is good practice to confirm it is there and can be read correctly. 
The next section will load the data as part of the upload to bigquery

In [4]:
# ALL File Paths should be declared at the TOP of the notebook
PATH_TO_CLEAN_CBC_DATA = "../data/Cloud_Data/1.0-rec-initial-data-cleaning.txt"

In [5]:
clean_data = pd.read_csv(PATH_TO_CLEAN_CBC_DATA, encoding = "ISO-8859-1", sep="\t")

/Users/rcdebaca/.pyenv/versions/funhacks371/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
clean_data.head()

,Unnamed: 0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,...,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial
0,2,Pacific Grove,US-CA,36.616700,-121.916700,1901,12/25/00,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Pueblo,US-CO,38.175251,-104.519575,1901,12/25/00,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Bristol,US-CT,41.671800,-72.949500,1901,12/25/00,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,Norwalk,US-CT,41.116700,-73.400000,1901,12/25/00,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,Glen Ellyn,US-IL,41.883300,-88.066700,1901,12/25/00,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Push this data up to bigQuery

In [7]:
# Prep creating a Table 
# table_id = "your-project.your_dataset.your_table_name"
table_id = project + "."+ source_dataset_id + "." + "rec_initial_data_cleaning"

print(table_id)             

# Create the Schema, and by that i mean DONT create a shcema cuz biqQuery does a pretty good job 
schema = []

birdproject-2020.audubon_cdc.rec_initial_data_cleaning


In [16]:
# NOTE: This ran originally, but then started presenting with the error:
#'AttributeError: 'str' object has no attribute 'project''
# If that happens you can create the table manually in bigQuery (defeats the purpose of automation,
# but if it fits it ships)

# Create the BigQuery Table 
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

AttributeError: 'str' object has no attribute 'project'

In [ ]:
#Send the File up to the Cloud

dataset_ref = client.dataset(source_dataset_id)
table_ref = dataset_ref.table("rec_initial_data_cleaning")

job_config = bigquery.LoadJobConfig()
job_config.field_delimiter='\t' 
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(PATH_TO_CLEAN_CBC_DATA, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, source_dataset_id, table_id))

## Build the Query and Submit it 

In [ ]:
# query = f"""
# WITH circles_hash as (SELECT x.*, ST_GEOHASH(ST_GEOGPOINT(x.lon,x.lat), 4) as geohash_circle, ST_GEOHASH(ST_GEOGPOINT(x.lon,x.lat), 7) as circle_id

# FROM `{project}.audubon_cdc.rec_initial_data_cleaning` x),

# stations_hash as (SELECT y.*, ST_GEOHASH(ST_GEOGPOINT(y.longitude,y.latitude),4) as geohash_station FROM `bigquery-public-data`.ghcn_d.ghcnd_stations y),

# circle_with_matched_stations as (SELECT * FROM circles_hash x INNER JOIN stations_hash y ON x.geohash_circle = y.geohash_station)

# SELECT x.*, y.temp_min_value,y.temp_max_value,y.precipitation_value,y.temp_avg,y.snow,y.snwd

# FROM circle_with_matched_stations x
# LEFT JOIN `{project}.audubon_cdc.flatten_noaa_from_1900_to_present` y ON x.id = y.id AND x.count_date = y.date

# ORDER BY circle_id DESC,count_date ASC """

# # Queries BigQuery public data set and creates a new dataframe object
# df_circles_to_stations_weather_data = client.query(query)


In [22]:
query = f"""
WITH circles_hash as (SELECT x.*, ST_GEOHASH(ST_GEOGPOINT(x.lon,x.lat), 4) as geohash_circle, ST_GEOHASH(ST_GEOGPOINT(x.lon,x.lat), 7) as circle_id

FROM `{project}.audubon_cdc.rec_initial_data_cleaning` x),

stations_hash as (SELECT y.*, ST_GEOHASH(ST_GEOGPOINT(y.longitude,y.latitude),4) as geohash_station FROM `bigquery-public-data`.ghcn_d.ghcnd_stations y),

circle_with_matched_stations as (SELECT * FROM circles_hash x INNER JOIN stations_hash y ON x.geohash_circle = y.geohash_station)

SELECT x.*, y.temp_min_value,y.temp_max_value,y.precipitation_value,y.temp_avg,y.snow,y.snwd

FROM circle_with_matched_stations x
LEFT JOIN `{project}.audubon_cdc.flatten_noaa_from_1900_to_present` y ON x.id = y.id AND x.count_date = y.date

ORDER BY circle_id DESC,count_date ASC """

# Queries BigQuery public data set and creates a new dataframe object
df_circles_to_stations_weather_data = client.query(query)


In [23]:
df_circles_to_stations_weather_data = df_circles_to_stations_weather_data.to_dataframe()

In [24]:
df_circles_to_stations_weather_data.shape

(1047595, 66)

In [26]:
# Saving stations in csv COMPRESSED IN GZIP!!!
df_circles_to_stations_weather_data.to_csv(r'1.1-circles_to_many_stations_usa_weather_data_' + str(time_now) +  '.csv', compression = "gzip")



## Top 5 records
Showing the top 5 records of the data extracted to the query above

In [27]:
df_circles_to_stations_weather_data.head()

,int64_field_0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,...,gsn_flag,hcn_crn_flag,wmoid,geohash_station,temp_min_value,temp_max_value,precipitation_value,temp_avg,snow,snwd
0,28806,Amchitka Island,US-AK,51.409713,179.284881,1977,1977-01-01,4.0,None,None,...,,,NaN,zcpk,NaN,NaN,NaN,NaN,NaN,NaN
1,28806,Amchitka Island,US-AK,51.409713,179.284881,1977,1977-01-01,4.0,None,None,...,,,NaN,zcpk,NaN,NaN,NaN,NaN,NaN,NaN
2,30036,Amchitka Island,US-AK,51.409713,179.284881,1978,1977-12-29,5.0,None,None,...,,,NaN,zcpk,NaN,NaN,NaN,NaN,NaN,NaN
3,30036,Amchitka Island,US-AK,51.409713,179.284881,1978,1977-12-29,5.0,None,None,...,,,NaN,zcpk,NaN,NaN,NaN,NaN,NaN,NaN
4,31321,Amchitka Island,US-AK,51.409713,179.284881,1979,1978-12-30,2.0,None,None,...,,,NaN,zcpk,NaN,NaN,NaN,NaN,NaN,NaN


## Statistics on dataset
How many records are empty for the various temperature measurements

In [28]:
import numpy as np

record_count = len(df_circles_to_stations_weather_data.index)
print('How many rows in dataset with missing vals: ', record_count)

temp_min_nas = df_circles_to_stations_weather_data.temp_min_value.isna().sum()
print("Missing min temperature: " + str(temp_min_nas))

temp_max_nas = df_circles_to_stations_weather_data.temp_max_value.isna().sum()
print("Missing max temperature: " + str(temp_max_nas))

temp_avg_nas = df_circles_to_stations_weather_data.temp_avg.isna().sum()
print("Missing avg temperature: " + str(temp_avg_nas))

snow = df_circles_to_stations_weather_data.snow.isna().sum()
print("Missing snow temperature: " + str(snow))

How many rows in dataset with missing vals:  1047595
Missing min temperature: 981842
Missing max temperature: 981834
Missing avg temperature: 1039022
Missing snow temperature: 955619


## Remove empty min/max temperature
Create new data frame

In [29]:
ref=df_circles_to_stations_weather_data.temp_min_value
paired_data=df_circles_to_stations_weather_data[ref.notna()]
paired_data.head()

,int64_field_0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,...,gsn_flag,hcn_crn_flag,wmoid,geohash_station,temp_min_value,temp_max_value,precipitation_value,temp_avg,snow,snwd
7,32617,Amchitka Island,US-AK,51.409713,179.284881,1980,1979-12-18,4.0,None,None,...,,,NaN,zcpk,-17.0,17.0,5.0,NaN,3.0,0.0
27,90930,Caribou,US-ME,46.912573,-67.947428,2012,2011-12-28,10.0,3.0,1.0,...,GSN,,72712.0,f2rd,-83.0,78.0,71.0,NaN,8.0,25.0
30,93245,Caribou,US-ME,46.912573,-67.947428,2013,2012-12-29,10.0,4.0,2.0,...,GSN,,72712.0,f2rd,-139.0,-61.0,0.0,NaN,0.0,229.0
32,95653,Caribou,US-ME,46.912573,-67.947428,2014,2014-01-01,7.0,5.0,1.0,...,GSN,,72712.0,f2rd,-282.0,-155.0,0.0,NaN,3.0,460.0
33,98110,Caribou,US-ME,46.912573,-67.947428,2015,2014-12-14,10.0,6.0,1.0,...,GSN,,72712.0,f2rd,0.0,22.0,0.0,NaN,0.0,150.0


In [31]:
paired_data.shape

(65753, 66)

## Size of dataframe

In [32]:
print("The total number of records in this data set is: ", len(paired_data.circle_name))

The total number of records in this data set is:  65753


## Only Data in the USA
Create new data frame for stations only for stations located in the USA

In [33]:
paired_data_usa = paired_data[paired_data.id.str.slice(stop=2)=="US"]
print("The number of rows station usa weather data: ", len(paired_data_usa))

The number of rows station usa weather data:  65539


In [34]:
# Saving stations in csv COMPRESSED IN GZIP!!!
paired_data_usa.to_csv(r'1.1-circles_to_many_stations_usa_weather_data_' + str(time_now) +  '.csv', compression = "gzip")

